In [77]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [78]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [79]:
class ConfigurationManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [80]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [88]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig) -> None:
        self.config = config
    
    def download_file(self)->str:
        """Fetch data from the URL
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Function returns None
        """
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path,exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
                zip_ref.extractall(unzip_path)
        except Exception as e:
            raise e


In [89]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-22 10:53:10,651] [INFO] [common] [yaml file:config\config.yaml loaded successfully]
[2024-03-22 10:53:10,651] [INFO] [common] [yaml file:params.yaml loaded successfully]
[2024-03-22 10:53:10,651] [INFO] [common] [created directory at artifacts]
[2024-03-22 10:53:10,651] [INFO] [common] [created directory at artifacts/data_ingestion]
[2024-03-22 10:53:10,659] [INFO] [1520354311] [Downloading data from https://drive.google.com/file/d/1yHEnuk12QwBO_VP54OEAUAZX6I26Mzag/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1yHEnuk12QwBO_VP54OEAUAZX6I26Mzag
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1yHEnuk12QwBO_VP54OEAUAZX6I26Mzag&confirm=t&uuid=19873ba5-ab58-43d5-9d06-6d130fbeb906
To: c:\Users\Prince\Desktop\Projects\Chest_Cancer_Classification\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:56<00:00, 861kB/s] 

[2024-03-22 10:54:12,104] [INFO] [1520354311] [Downloaded data from https://drive.google.com/file/d/1yHEnuk12QwBO_VP54OEAUAZX6I26Mzag/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
